In [ ]:
from google.colab import drive
import sys
import torch

drive.mount('/content/drive')

sys.path.insert(1,'/content/drive/My Drive/Mestrado/Codigo/Imports/')

Mounted at /content/drive


Select the enhancement algorithm and the dataset folder.

In [ ]:
dataset_folder = "/content/drive/My Drive/Mestrado/Codigo/Datasets/TestCXR/"
dataset = 'TestCXR'
enhancement = None
#enhancement = 'clahe'
#enhancement = 'um'
#enhancement = 'hef'
#enhancement = 'atace'
#enhancement = 'tcdhe'

file_folder = [dataset_folder+"images/Viral Pneumonia/",
               dataset_folder+"images/Normal/",
               dataset_folder+"images/Lung_Opacity/",
               dataset_folder+"images/COVID/"]

Create the dataloader object.

In [ ]:
from data_loader import folder_data_loader

data_loader = folder_data_loader(class_path_list=file_folder, img_size=(224,224), batch_size=32, train_ratio = 0.8, dataset_size_scaling = 1.0, ld_net=enhancement)

Create a custom dataloader in a keras format.

In [ ]:
import numpy as np
import tensorflow as tf
import PIL
from inference import image_haze_removal
import os
import math
from torchvision.transforms import ToPILImage
from enhancement_algorithms import UM,HEF,CLAHE,ATACE,TCDHE,TCDHE

class CustomDataLoader(tf.keras.utils.Sequence):

  def __init__(self, folder_data_loader, mode='train'):
    self.folder_data_loader = folder_data_loader
    self.mode = mode
    self.class_representation = folder_data_loader.class_representation

  def __len__(self):
    return self.folder_data_loader.__len__(mode=self.mode)

  def __getitem__(self, idx):
    x,y = self.folder_data_loader.get_item(idx,mode=self.mode)
    batch_x = []
    to_pil = ToPILImage(mode=None)
    for i in range(0,x.shape[0]):
      image = x[i]
      image = to_pil(image*255)
      image = image.convert('RGB')
      image = np.asarray(image)
      image = (image/255.0).astype(np.single)
      batch_x.append(image)
    batch_x = np.asarray(batch_x)
    batch_y = y
    return batch_x,batch_y

Create the train and validation dataloader.

In [ ]:
train_dataloader = CustomDataLoader(data_loader,mode='train')

validation_dataloader = CustomDataLoader(data_loader,mode='validation')

Create the model using transfer learning.

In [ ]:
#from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from keras import applications
from keras.optimizers import SGD
import tensorflow as tf

batch_size = 32

#base_model = applications.vgg19.VGG19(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
#classification_folder = '/content/drive/My Drive/Mestrado/Codigo/Classification/TestCXR/other_cnns/vgg/'

#base_model = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
#classification_folder = '/content/drive/My Drive/Mestrado/Codigo/Classification/TestCXR/other_cnns/densenet/'

#base_model = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
#classification_folder = '/content/drive/My Drive/Mestrado/Codigo/Classification/TestCXR/other_cnns/resnet/'

#base_model = tf.keras.applications.nasnet.NASNetLarge(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
#classification_folder = '/content/drive/My Drive/Mestrado/Codigo/Classification/TestCXR/other_cnns/nasnet/'

#base_model = tf.keras.applications.xception.Xception(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
#classification_folder = '/content/drive/My Drive/Mestrado/Codigo/Classification/TestCXR/other_cnns/xception/'

#base_model = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
#classification_folder = '/content/drive/My Drive/Mestrado/Codigo/Classification/TestCXR/other_cnns/inception/'

#base_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
#classification_folder = '/content/drive/My Drive/Mestrado/Codigo/Classification/TestCXR/other_cnns/inceptionresnet/'

base_model = applications.mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
classification_folder = '/content/drive/My Drive/Mestrado/Codigo/Classification/TestCXR/other_cnns/mobilenet/'

base_model.trainable = False
x = base_model.input
x = base_model(x, training=False)

#x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Flatten()(x)
x = Dense(128, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)

predictions = Dense(4, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4,momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.CategoricalCrossentropy(),
                       tf.keras.metrics.Precision()]
              )

#model.summary()

Run the training, save the train and finetune history, and the confusion matrix.

In [ ]:
class_weight = {}
class_representation = train_dataloader.class_representation
for i in range(0,class_representation.shape[0]):
  class_weight[i] = class_representation.sum()/(class_representation[i]*class_representation.shape[0])

print(class_weight)

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1,restore_best_weights=True)]

history1=model.fit(
    train_dataloader,
    validation_data = validation_dataloader,
    batch_size = 32,
    epochs = 20,
    class_weight=class_weight,
    callbacks = callbacks)

model.trainable = True
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6,momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.CategoricalCrossentropy(),
                       tf.keras.metrics.Precision()]
              )
history2=model.fit(
    train_dataloader,
    validation_data = validation_dataloader,
    batch_size = 32,
    epochs = 10,
    class_weight = class_weight,
    callbacks = callbacks)

import pickle

if ld_net == None:
  ld_net = 'None'

test_folder = classification_folder

with open(test_folder+ld_net+'/trainHistoryDict', 'wb') as file_pi:
  pickle.dump(history1.history, file_pi)

with open(test_folder+ld_net+'/finetuneHistoryDict', 'wb') as file_pi:
  pickle.dump(history2.history, file_pi)

model.save_weights(test_folder+ld_net+'/weight'+'.h5')

import pandas as pd
from sklearn.metrics import confusion_matrix

y_pred = (model.predict(x=validation_dataloader,batch_size=32)).argmax(axis=1)

y_true = []

for i in range(0,validation_dataloader.__len__()):
  batch_x,batch_y = validation_dataloader.__getitem__(i)
  for j in range(0,batch_y.shape[0]):
    y_true.append(batch_y[j])

y_true = np.asarray(y_true).argmax(axis=1)

conf_matrix = confusion_matrix(y_true[0:y_pred.shape[0]], y_pred)
df = pd.DataFrame(conf_matrix)
df.to_csv(test_folder+ld_net+'/confusion_matrix'+'.csv')